In [1]:
using Pkg; Pkg.activate("/home/dhairyagandhi96/modelzoo/model-zoo/script/../vision/mnist"); Pkg.instantiate(); Pkg.add(PackageSpec(name="CUDAdrv", rev = "master")); Pkg.add(PackageSpec(name="CUDAnative", rev = "master")); Pkg.add(PackageSpec(name="CuArrays", rev = "master")); Pkg.add(PackageSpec(name="Flux", rev = "master"))

using Flux, Flux.Data.MNIST, Statistics
using Flux: onehotbatch, onecold, crossentropy, throttle
using Base.Iterators: repeated
using CuArrays

  Updating registry at `~/.julia/registries/General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`
  Updating git-repo `https://github.com/JuliaGPU/CUDAdrv.jl.git`
 Resolving package versions...
  Updating `~/modelzoo/model-zoo/vision/mnist/Project.toml`
 [no changes]
  Updating `~/modelzoo/model-zoo/vision/mnist/Manifest.toml`
 [no changes]
  Updating git-repo `https://github.com/JuliaGPU/CUDAnative.jl.git`
 Resolving package versions...
  Updating `~/modelzoo/model-zoo/vision/mnist/Project.toml`
 [no changes]
  Updating `~/modelzoo/model-zoo/vision/mnist/Manifest.toml`
 [no changes]
  Updating git-repo `https://github.com/JuliaGPU/CuArrays.jl.git`
 Resolving package versions...
  Updating `~/modelzoo/model-zoo/vision/mnist/Project.toml`
 [no changes]
  Updating `~/modelzoo/model-zoo/vision/mnist/Manifest.toml`
 [no changes]
  Updating git-repo `https://github.com/FluxML/Flux.jl.git`
 Resolving package versions...
  Updating `~/modelzoo/model-zoo/vision/mnist/Pr

Classify MNIST digits with a simple multi-layer-perceptron

In [2]:
imgs = MNIST.images()

┌ Info: Downloading MNIST dataset
└ @ Flux.Data.MNIST /home/dhairyagandhi96/.julia/packages/Flux/eovHg/src/data/mnist.jl:23
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   469  100   469    0     0    905      0 --:--:-- --:--:-- --:--:--   903
100 9680k  100 9680k    0     0  10.2M      0 --:--:-- --:--:-- --:--:-- 10.2M
┌ Info: Downloading MNIST dataset
└ @ Flux.Data.MNIST /home/dhairyagandhi96/.julia/packages/Flux/eovHg/src/data/mnist.jl:23
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   469  100   469    0     0   5043      0 --:--:-- --:--:-- --:--:--  5043
100 28881  100 28881    0     0   122k      0 --:--:-- --:--:-- --:--:-- 11.9M
┌ Info: Downloading MNIST dataset
└ @ Flux.Data.MNIST /home/dhairyagandhi96/.julia/packages/Flux/eovHg/src/data/mnist.jl

60000-element Array{Array{ColorTypes.Gray{FixedPointNumbers.Normed{UInt8,8}},2},1}:
 [Gray{N0f8}(0.0) Gray{N0f8}(0.0) … Gray{N0f8}(0.0) Gray{N0f8}(0.0); Gray{N0f8}(0.0) Gray{N0f8}(0.0) … Gray{N0f8}(0.0) Gray{N0f8}(0.0); … ; Gray{N0f8}(0.0) Gray{N0f8}(0.0) … Gray{N0f8}(0.0) Gray{N0f8}(0.0); Gray{N0f8}(0.0) Gray{N0f8}(0.0) … Gray{N0f8}(0.0) Gray{N0f8}(0.0)]
 [Gray{N0f8}(0.0) Gray{N0f8}(0.0) … Gray{N0f8}(0.0) Gray{N0f8}(0.0); Gray{N0f8}(0.0) Gray{N0f8}(0.0) … Gray{N0f8}(0.0) Gray{N0f8}(0.0); … ; Gray{N0f8}(0.0) Gray{N0f8}(0.0) … Gray{N0f8}(0.0) Gray{N0f8}(0.0); Gray{N0f8}(0.0) Gray{N0f8}(0.0) … Gray{N0f8}(0.0) Gray{N0f8}(0.0)]
 [Gray{N0f8}(0.0) Gray{N0f8}(0.0) … Gray{N0f8}(0.0) Gray{N0f8}(0.0); Gray{N0f8}(0.0) Gray{N0f8}(0.0) … Gray{N0f8}(0.0) Gray{N0f8}(0.0); … ; Gray{N0f8}(0.0) Gray{N0f8}(0.0) … Gray{N0f8}(0.0) Gray{N0f8}(0.0); Gray{N0f8}(0.0) Gray{N0f8}(0.0) … Gray{N0f8}(0.0) Gray{N0f8}(0.0)]
 [Gray{N0f8}(0.0) Gray{N0f8}(0.0) … Gray{N0f8}(0.0) Gray{N0f8}(0.0); Gray{N0f8}(0.0) Gray{N0f8

Stack images into one large batch

In [3]:
X = hcat(float.(reshape.(imgs, :))...) |> gpu

labels = MNIST.labels()

60000-element Array{Int64,1}:
 5
 0
 4
 1
 9
 2
 1
 3
 1
 4
 ⋮
 2
 9
 5
 1
 8
 3
 5
 6
 8

One-hot-encode the labels

In [4]:
Y = onehotbatch(labels, 0:9) |> gpu

m = Chain(
  Dense(28^2, 32, relu),
  Dense(32, 10),
  softmax) |> gpu

loss(x, y) = crossentropy(m(x), y)

accuracy(x, y) = mean(onecold(m(x)) .== onecold(y))

dataset = repeated((X, Y), 200) |> gpu
evalcb = () -> @show(loss(X, Y))
opt = ADAM(params(m))

@info "starting training.."
Flux.train!(loss, dataset, opt, cb = throttle(evalcb, 10))

accuracy(X, Y)

┌ Warning: ADAM(params) is deprecated; use ADAM(η::Float64) instead
│   caller = top-level scope at none:0
└ @ Core none:0
┌ Info: starting training..
└ @ Main.##360 string:16
┌ Warning: train!(loss, data, opt) is deprecated; use train!(loss, params, data, opt) instead
│   caller = ip:0x0
└ @ Core :-1
┌ Warning: calls to Base intrinsics might be GPU incompatible
│   exception = (CUDAnative.MethodSubstitutionWarning(exp(x::T) where T<:Union{Float32, Float64} in Base.Math at special/exp.jl:75, exp(x::Float32) in CUDAnative at /home/dhairyagandhi96/.julia/packages/CUDAnative/Mt0fi/src/device/libdevice.jl:90), Base.StackTraces.StackFrame[exp at exp.jl:75, mapreducedim_kernel_parallel at mapreduce.jl:29])
└ @ CUDAnative /home/dhairyagandhi96/.julia/packages/CUDAnative/Mt0fi/src/compiler/irgen.jl:102
┌ Warning: calls to Base intrinsics might be GPU incompatible
│   exception = (CUDAnative.MethodSubstitutionWarning(exp(x::T) where T<:Union{Float32, Float64} in Base.Math at special/exp.jl:75, 

0.9235333333333333

Test set accuracy

In [5]:
tX = hcat(float.(reshape.(MNIST.images(:test), :))...) |> gpu
tY = onehotbatch(MNIST.labels(:test), 0:9) |> gpu

accuracy(tX, tY)

0.9262